## Install


Clone this repo.

- In a command line or shell, navigate to the cloned repo.
- `pip install -e ./cobrapy`


In [ ]:
%pip install -e ../.

## Standard Example


### Step #0 - Load Environment Variables


This utility depends on connections to outside services. You will neeed to make sure the following environment variables are set.

```
AZURE_OPENAI_GPT4_VISION_API_KEY=""
AZURE_OPENAI_GPT4_VISION_ENDPOINT=""
AZURE_OPENAI_GPT4_VISION_API_VERSION="2024-06-01"
AZURE_OPENAI_GPT4_VISION_DEPLOYMENT=""


AZURE_OPENAI_WHISPER_API_KEY=""
AZURE_OPENAI_WHISPER_ENDPOINT=""
AZURE_OPENAI_WHISPER_API_VERSION="2024-06-01"
AZURE_OPENAI_WHISPER_DEPLOYMENT=""
AZURE_OPENAI_WHISPER_DEPLOYMENT="whisper-001"
```

One way to do this is to create a `.env` file and use a utility such as python-dotenv to load it. You can copy or rename the `sample.env` file and fill it with your own values.


### Step #1 - Create a VideoClient object


In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd()))

from cobrapy import VideoClient

client = VideoClient(video_path="path/to/your/video.mp4")
client.manifest.model_dump()

### Step #2 - Preprocess Video


In [ ]:
# client.preprocess_video(overwrite_output=True)
# client.manifest.model_dump()
client.preprocess_video(
    # output_directory="./output_dir",  # control where to save the manifest and other files
    segment_length=10,  # how long should each segment be in seconds
    fps=1,  # how many frames per second to sample from the video (i.e. 1 = 1 frame per second, 1/3 = 1 frame per 3 seconds)
    max_workers=5,  # how many threads to use for processing. Default is to use number of cores minus 1.
    allow_partial_segments=True,  # if False, the last segment will be discarded if it is shorter than segment_length
    overwrite_output=True,  # any files in a directory with the same name will be overwritten
)

### Step #3 - Run an Analysis


In [ ]:
from cobrapy.analysis import BasicSummary, ActionSummary

action_config = ActionSummary()
# action_config.system_prompt_lens="Your lens here"
# action_config.results_template=[{{"start_timestamp":"0s","end_timestamp":"10s","actions":"Your action here","summary":"description"}}]
action_summary = client.analyze_video(
    analysis_config=action_config, run_async=True, max_concurrent_tasks=10
)
action_summary

## Configurability


### Load client from existing manifest


In [ ]:
from cobrapy import VideoClient

client = VideoClient(manifest="./path_to_your_video/_video_manifest.json")

### Pre-process video with custom parameters


In [ ]:
client.preprocess_video(
    output_directory="./output_dir",  # control where to save the manifest and other files
    segment_length=20,  # how long should each segment be in seconds
    fps=1,  # how many frames per second to sample from the video (i.e. 1 = 1 frame per second, 1/3 = 1 frame per 3 seconds)
    max_workers=10,  # how many threads to use for processing. Default is to use number of cores minus 1.
    allow_partial_segments=True,  # if False, the last segment will be discarded if it is shorter than segment_length
    overwrite_output=True,  # any files in a directory with the same name will be overwritten
)

### Parallel Analysis

Analyses that have the analysis_sequence value of "mapreduce" can be run in parallel using async patterns. Throughput and parrallelism will be constrained by the rate limits of your LLM endpoint.


In [ ]:
from cobrapy.analysis import BasicSummary

basic_config = BasicSummary()

basic_summary = client.analyze_video(basic_config, run_async=True)

### Customizing an existing analysis configuration


In [ ]:
from cobrapy.analysis import BasicSummary

# Load the default configuration
customizable_config = BasicSummary()

# Output the config values
customizable_config.model_dump()

In [ ]:
# Customize the configuration
customizable_config.lens_prompt = "Analyze the video from the perspective of a wildlife documentarian. Describe any behaviors -- whether from animals, humans or anything else as though you are preparing for a Discovery Channel documentary. In addition to the normal summary, provide a draft script of a narration for the video."

customizable_config.results_template = {
    "segment_summary": "Text summary of what is happening in the clip segment.",
    "narration": "Draft narration script for the segment. In the style of Morgan Freeman or David Attenborough.",
}

customizable_config.run_final_summary = True

customizable_config.summary_prompt = "This video is from a comedy prank television show. Your job is to summarize the video based on the provided segment summaries. Explain the joke. Your summary should 2-3 sentences max."

In [ ]:
documentary_summary = client.analyze_video(customizable_config, run_async=True)
documentary_summary

In [ ]:
from pprint import pprint

pprint(client.manifest.final_summary)

### Running a sequential / refine analysis


In [ ]:
from cobrapy import VideoClient

client = VideoClient(video_path="C:\\videos\\PCL2.mp4")

manifest = client.preprocess_video(
    segment_length=30,
    fps=0.33,
    overwrite_output=True,
)

In [ ]:
from cobrapy.analysis.chapter_summary import ChapterAnalysis

chapter_config = ChapterAnalysis()
chapter_summary = client.analyze_video(chapter_config)